In [1]:
#ENV["JULIA_NUM_THREADS"] = "4" # Itt állítsd be a kívánt szálak számát
using Base.Threads
@info "A szkript futási környezete" nthreads()

┌ Info: A szkript futási környezete
└   nthreads() = 4


In [2]:
using LowLevelFEM, LinearAlgebra
gmsh.initialize()
gmsh.open("beam.geo")

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (2), incompatible header (12), dep missing source (4))


Info    : Reading 'beam.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000887878s, CPU 0.000888s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000870665s, CPU 0.00087s)
Info    : Meshing 3D..

In [3]:
mat = material("body", E=2e5, ν=0.3)
cantilever = Problem([mat], type=:Solid)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 386)


Problem("beam", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 6561)

In [4]:
force = load("top", fy=-1)
supp = displacementConstraint("left", ux=0, uy=0, uz=0)

("left", 0, 0, 0)

In [5]:
K = stiffnessMatrix(cantilever)

19683×19683 SparseArrays.SparseMatrixCSC{Float64, Int64} with 3143378 stored entries:
⎡⡿⣯⣿⠿⠿⢿⣬⣤⣄⣿⠛⣥⣽⡿⢈⣹⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣶⣾⣿⎤
⎢⣿⡟⢿⣷⡲⠾⠙⠉⠉⣿⡮⠁⢹⡿⠘⠻⢿⡻⠽⢿⣿⣟⣛⣛⣛⡛⠛⠛⠛⠛⠛⠻⠿⠭⠭⠭⠭⠭⠭⠿⎥
⎢⣿⣇⣸⡎⠻⣦⣀⡀⠀⣇⢧⡀⣸⣹⢀⣀⣈⠻⣄⣀⣀⣀⣀⡈⠉⠉⠉⠉⠛⠓⠒⠒⠶⠦⠤⠤⣤⣄⣀⣀⎥
⎢⠂⣿⡗⠀⠀⠸⡿⣯⡙⢺⠀⢿⡗⡇⢸⣿⠺⣍⠓⠒⠒⠾⠿⠭⠭⢭⣭⣉⣉⣉⣉⠉⠉⠉⠉⠉⠙⠛⠓⠒⎥
⎢⣤⣽⣧⣤⠤⢤⣳⣈⣻⣾⣤⣼⣷⠇⠀⢿⣤⣼⣷⣤⣤⣤⣤⣤⣤⣤⣤⣤⣤⣬⣭⣭⣭⣽⣿⣶⣶⣶⣶⣤⎥
⎢⠟⣤⠎⠋⠉⠳⣤⣄⣀⣿⢿⣷⡄⠀⠀⠰⢾⣻⡷⠶⠾⠿⢿⣿⣛⣛⣛⣛⣛⣿⣭⣭⣭⣭⣭⣽⣷⣶⣶⠶⎥
⎢⣷⡿⣷⡶⣖⣺⠽⠭⠽⠟⠀⠉⢻⣶⣶⢾⣿⣿⣿⣿⣿⠿⠿⠿⠿⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣆⣰⣶⡀⠀⢰⣶⣶⣤⣄⢀⡀⣸⣟⢿⣷⣾⣿⣷⡾⠿⠿⠿⢿⣟⣛⣛⣛⣿⣯⣭⣭⣭⣭⣿⣿⣶⣶⣶⣶⎥
⎢⣿⣿⣿⡳⣦⡘⡞⢦⣀⣿⣾⣳⣿⣿⣾⣿⠻⣦⡘⠛⠒⠒⠒⠶⠶⠤⠤⠤⣤⣤⣀⣀⣀⣀⣀⠀⠀⠀⠈⠉⎥
⎢⣿⣿⣷⣇⠀⢹⢹⠀⠙⣿⢹⡏⣿⣿⣹⡿⣶⠈⢻⣶⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠉⠛⠓⎥
⎢⣿⣿⣿⢿⠀⢸⣸⡄⠀⣿⣾⡇⣿⡟⣿⡇⢸⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⢸⡀⠸⡟⡇⠀⣿⣿⣷⣿⡇⣿⣇⢸⡄⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠸⡇⠀⡇⣇⠀⣿⣿⢸⣿⡇⣿⢹⠘⡇⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠀⡇⠀⡇⢻⠀⣿⣿⢸⣿⣿⣿⢸⠀⡇⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠀⢿⠀⡇⢸⡀⣿⣿⣼⣿⣿⡿⣿⠀⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⡀⢸⠀⡇⠘⡇⣿⡇⣿⣿⣿⡇⣿⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⡟⡇⠸⡇⡇⠀⣇⣿⡇⣿⣿⣿⡇⣿⠀⢸⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⎥
⎢⣿⣿⡇⡇⠀⡇⡇⠀⢻⣿⣇⣿⣿⣿⣿⣿⠀⠘⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⎥
⎢⢻⣿⡇⡇⠀⢿⣷⠀⢸⣿⢹⣿⣿⣿⢸⣿⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⎥
⎣⣾⣿⣧⡇⠀⢸⢹⠀⠘⣿⢸⡟⣿⣿⢸⣿⡆⠀⢿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⎦

In [6]:
using BenchmarkTools

In [7]:
@time K = LowLevelFEM.stiffnessMatrixSolid(cantilever)

  3.801930 seconds (20.09 M allocations: 5.206 GiB, 41.88% gc time, 0.28% compilation time)


19683×19683 SparseArrays.SparseMatrixCSC{Float64, Int64} with 3143378 stored entries:
⎡⡿⣯⣿⠿⠿⢿⣬⣤⣄⣿⠛⣥⣽⡿⢈⣹⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣶⣾⣿⎤
⎢⣿⡟⢿⣷⡲⠾⠙⠉⠉⣿⡮⠁⢹⡿⠘⠻⢿⡻⠽⢿⣿⣟⣛⣛⣛⡛⠛⠛⠛⠛⠛⠻⠿⠭⠭⠭⠭⠭⠭⠿⎥
⎢⣿⣇⣸⡎⠻⣦⣀⡀⠀⣇⢧⡀⣸⣹⢀⣀⣈⠻⣄⣀⣀⣀⣀⡈⠉⠉⠉⠉⠛⠓⠒⠒⠶⠦⠤⠤⣤⣄⣀⣀⎥
⎢⠂⣿⡗⠀⠀⠸⡿⣯⡙⢺⠀⢿⡗⡇⢸⣿⠺⣍⠓⠒⠒⠾⠿⠭⠭⢭⣭⣉⣉⣉⣉⠉⠉⠉⠉⠉⠙⠛⠓⠒⎥
⎢⣤⣽⣧⣤⠤⢤⣳⣈⣻⣾⣤⣼⣷⠇⠀⢿⣤⣼⣷⣤⣤⣤⣤⣤⣤⣤⣤⣤⣤⣬⣭⣭⣭⣽⣿⣶⣶⣶⣶⣤⎥
⎢⠟⣤⠎⠋⠉⠳⣤⣄⣀⣿⢿⣷⡄⠀⠀⠰⢾⣻⡷⠶⠾⠿⢿⣿⣛⣛⣛⣛⣛⣿⣭⣭⣭⣭⣭⣽⣷⣶⣶⠶⎥
⎢⣷⡿⣷⡶⣖⣺⠽⠭⠽⠟⠀⠉⢻⣶⣶⢾⣿⣿⣿⣿⣿⠿⠿⠿⠿⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣆⣰⣶⡀⠀⢰⣶⣶⣤⣄⢀⡀⣸⣟⢿⣷⣾⣿⣷⡾⠿⠿⠿⢿⣟⣛⣛⣛⣿⣯⣭⣭⣭⣭⣿⣿⣶⣶⣶⣶⎥
⎢⣿⣿⣿⡳⣦⡘⡞⢦⣀⣿⣾⣳⣿⣿⣾⣿⠻⣦⡘⠛⠒⠒⠒⠶⠶⠤⠤⠤⣤⣤⣀⣀⣀⣀⣀⠀⠀⠀⠈⠉⎥
⎢⣿⣿⣷⣇⠀⢹⢹⠀⠙⣿⢹⡏⣿⣿⣹⡿⣶⠈⢻⣶⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠉⠛⠓⎥
⎢⣿⣿⣿⢿⠀⢸⣸⡄⠀⣿⣾⡇⣿⡟⣿⡇⢸⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⢸⡀⠸⡟⡇⠀⣿⣿⣷⣿⡇⣿⣇⢸⡄⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠸⡇⠀⡇⣇⠀⣿⣿⢸⣿⡇⣿⢹⠘⡇⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠀⡇⠀⡇⢻⠀⣿⣿⢸⣿⣿⣿⢸⠀⡇⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠀⢿⠀⡇⢸⡀⣿⣿⣼⣿⣿⡿⣿⠀⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⡀⢸⠀⡇⠘⡇⣿⡇⣿⣿⣿⡇⣿⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⡟⡇⠸⡇⡇⠀⣇⣿⡇⣿⣿⣿⡇⣿⠀⢸⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⎥
⎢⣿⣿⡇⡇⠀⡇⡇⠀⢻⣿⣇⣿⣿⣿⣿⣿⠀⠘⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⎥
⎢⢻⣿⡇⡇⠀⢿⣷⠀⢸⣿⢹⣿⣿⣿⢸⣿⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⎥
⎣⣾⣿⣧⡇⠀⢸⢹⠀⠘⣿⢸⡟⣿⣿⢸⣿⡆⠀⢿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⎦

In [8]:
@time K = LowLevelFEM.stiffnessMatrixSolidParallel(cantilever)

  8.913894 seconds (9.93 M allocations: 5.353 GiB, 18.49% gc time, 72.08% compilation time)


19683×19683 SparseArrays.SparseMatrixCSC{Float64, Int64} with 3143393 stored entries:
⎡⡿⣯⣿⠿⠿⢿⣬⣤⣄⣿⠛⣥⣽⡿⢈⣹⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣶⣾⣿⎤
⎢⣿⡟⢿⣷⡲⠾⠙⠉⠉⣿⡮⠁⢹⡿⠘⠻⢿⡻⠽⢿⣿⣟⣛⣛⣛⡛⠛⠛⠛⠛⠛⠻⠿⠭⠭⠭⠭⠭⠭⠿⎥
⎢⣿⣇⣸⡎⠻⣦⣀⡀⠀⣇⢧⡀⣸⣹⢀⣀⣈⠻⣄⣀⣀⣀⣀⡈⠉⠉⠉⠉⠛⠓⠒⠒⠶⠦⠤⠤⣤⣄⣀⣀⎥
⎢⠂⣿⡗⠀⠀⠸⡿⣯⡙⢺⠀⢿⡗⡇⢸⣿⠺⣍⠓⠒⠒⠾⠿⠭⠭⢭⣭⣉⣉⣉⣉⠉⠉⠉⠉⠉⠙⠛⠓⠒⎥
⎢⣤⣽⣧⣤⠤⢤⣳⣈⣻⣾⣤⣼⣷⠇⠀⢿⣤⣼⣷⣤⣤⣤⣤⣤⣤⣤⣤⣤⣤⣬⣭⣭⣭⣽⣿⣶⣶⣶⣶⣤⎥
⎢⠟⣤⠎⠋⠉⠳⣤⣄⣀⣿⢿⣷⡄⠀⠀⠰⢾⣻⡷⠶⠾⠿⢿⣿⣛⣛⣛⣛⣛⣿⣭⣭⣭⣭⣭⣽⣷⣶⣶⠶⎥
⎢⣷⡿⣷⡶⣖⣺⠽⠭⠽⠟⠀⠉⢻⣶⣶⢾⣿⣿⣿⣿⣿⠿⠿⠿⠿⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣆⣰⣶⡀⠀⢰⣶⣶⣤⣄⢀⡀⣸⣟⢿⣷⣾⣿⣷⡾⠿⠿⠿⢿⣟⣛⣛⣛⣿⣯⣭⣭⣭⣭⣿⣿⣶⣶⣶⣶⎥
⎢⣿⣿⣿⡳⣦⡘⡞⢦⣀⣿⣾⣳⣿⣿⣾⣿⠻⣦⡘⠛⠒⠒⠒⠶⠶⠤⠤⠤⣤⣤⣀⣀⣀⣀⣀⠀⠀⠀⠈⠉⎥
⎢⣿⣿⣷⣇⠀⢹⢹⠀⠙⣿⢹⡏⣿⣿⣹⡿⣶⠈⢻⣶⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠉⠛⠓⎥
⎢⣿⣿⣿⢿⠀⢸⣸⡄⠀⣿⣾⡇⣿⡟⣿⡇⢸⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⢸⡀⠸⡟⡇⠀⣿⣿⣷⣿⡇⣿⣇⢸⡄⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠸⡇⠀⡇⣇⠀⣿⣿⢸⣿⡇⣿⢹⠘⡇⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠀⡇⠀⡇⢻⠀⣿⣿⢸⣿⣿⣿⢸⠀⡇⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠀⢿⠀⡇⢸⡀⣿⣿⣼⣿⣿⡿⣿⠀⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⡀⢸⠀⡇⠘⡇⣿⡇⣿⣿⣿⡇⣿⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⡟⡇⠸⡇⡇⠀⣇⣿⡇⣿⣿⣿⡇⣿⠀⢸⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⎥
⎢⣿⣿⡇⡇⠀⡇⡇⠀⢻⣿⣇⣿⣿⣿⣿⣿⠀⠘⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⎥
⎢⢻⣿⡇⡇⠀⢿⣷⠀⢸⣿⢹⣿⣿⣿⢸⣿⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⎥
⎣⣾⣿⣧⡇⠀⢸⢹⠀⠘⣿⢸⡟⣿⣿⢸⣿⡆⠀⢿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⎦

In [9]:
f = loadVector(cantilever, [force])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :f3D, Problem("beam", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 6561))

In [10]:
applyBoundaryConditions!(K, f, [supp])

In [11]:
E = mat.E
ν = mat.ν

0.3

In [12]:
u = K \ f
A = (u ∘ ∇ + ∇ ∘ u) / 2
I = unitTensor(A)
S = E / (1 + ν) * (A + ν / (1 - 2ν) * trace(A) * I)

LowLevelFEM.TensorField([[27.193121633168918; 2.016696764190115; … ; -9.91958802744247; -155.8307275688762;;], [24.08775914321881; 3.2023985634093384; … ; -17.125824218797046; -149.49058269989882;;], [20.338453065085766; -0.2647528585928653; … ; -17.12582421869796; -149.490582699318;;], [30.95258796209376; -3.945277394425099; … ; -9.919588027145862; -155.83072756710882;;], [-0.1846066092127007; 3.053204275591378; … ; -1.635538742575326; -49.29667561574183;;], [-0.19607679658435564; 4.331598987996298; … ; -10.853573028716033; -46.414859809062754;;], [-0.20166402743159195; -0.2530089911918474; … ; -10.853573028643567; -46.41485980853351;;], [-0.18736351774844673; -5.2845782255454505; … ; -1.6355387423469263; -49.29667561406621;;], [-27.538100152426047; 2.306165016262653; … ; -1.7485705393306201; 48.928831514222225;;], [-24.46727726357855; 3.0806674027274177; … ; -10.96424604517092; 46.08950080191069;;]  …  [-0.13954817835389785; 0.002211603604063474; … ; -0.17269100433462073; -0.15096200

In [13]:
showDoFResults(u, :vector, visible=true)
Sz = showDoFResults(S, :sz)
Syz = showDoFResults(S, :syz)

3

In [14]:
plotOnPath("path", Sz)
plotOnPath("path", Syz)

5

In [15]:
b = -S ⋅ ∇

LowLevelFEM.VectorField([[-3.1150253143469726; 15.166420429597942; … ; 17.51393537397165; 18.991799199250664;;], [-3.801109710909875; 12.577647575806818; … ; 19.388198106093235; 12.869442588277536;;], [-0.12238473795816418; 11.778339270589305; … ; 19.388198106090464; 12.869442588219142;;], [6.816133505422173; 10.506215238452745; … ; 17.5139353739597; 18.991799199041086;;], [-0.5184677078670283; 24.357980345510775; … ; 15.917464958611871; 4.302861634888984;;], [-0.2785080478628156; 18.90109730461973; … ; 17.46480622059552; 1.2057605321027778;;], [0.003879692952268865; 17.839939177168898; … ; 17.464806220600348; 1.2057605320884899;;], [0.29115500849669634; 16.329848735559302; … ; 15.917464958637284; 4.3028616347578605;;], [3.7097855676230296; 22.62616545845858; … ; 15.903096544181107; -4.27773594289296;;], [2.9507415737986142; 15.937238376036694; … ; 17.45125252894832; -1.2062938585704148;;]  …  [0.002900495520867935; 0.011401274799406355; … ; -0.028961077969900016; 0.02966457727263834;;

In [16]:
showDoFResults(b, :vector)

6

In [17]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [18]:
gmsh.finalize()